In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from utilities.rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from utilities.data_preprocessors import get_random_data, MetricsAfterEachEpoch

In [8]:
global MAX_EPOCHS, MAX_BATCH_SIZE, architecture, label, keywords
MAX_EPOCHS, MAX_BATCH_SIZE = 15, 1024
architecture = 'LeNet5'
label, keywords = 'noisy_bl-experiments_test', 'benchmark_testing_test'

In [9]:
def model_extractor(activation_func):
    # Creating a LeNet5 Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))

    # #Instantiating Layer 2
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))

    # #Instantiating Layer 3
    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))

    model.add(Flatten())

    #Instantiating Layer 4
    model.add(Dense(84, activation=activation_func)) 

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [10]:
def diff_optimizer(curr_optimizer, activation_func, label=None, curr_epochs=MAX_EPOCHS):
    model = model_extractor(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, batch_size=1024).history
    score = model.evaluate(x_test, y_test, batch_size=MAX_BATCH_SIZE)
    
    #Saving the model
    if label == 'RAdam':
        model.save("../../saved_models_noisy/{}_{}_{}.hdf5".format('RAdam', activation_func, architecture))
    else: 
        model.save("../../saved_models_noisy/{}_{}_{}.hdf5".format(curr_optimizer, activation_func, architecture))
    return score, history

In [11]:
MAX_TRAINING = 1000
MAX_VALIDATION = 100
MAX_TESTING = 100

base = 'noisy_dataset/noisy/cifar_10_{}.npy'
x_train_max, y_train = np.load(base.format('x_train')), np.load(base.format('y_train'))
x_test_max, y_test = np.load(base.format('x_test')), np.load(base.format('y_test'))
y_train_max, y_test_max = to_categorical(y_train, num_classes=10), to_categorical(y_test, num_classes=10)

train_cutoff = int(x_train_max.shape[0] - MAX_VALIDATION*4)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, train_cutoff, MAX_TRAINING)
x_val, y_val = get_random_data(x_train_max, y_train_max, train_cutoff+1, x_train_max.shape[0], MAX_VALIDATION)
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=1000, Y=1000
Number of Validation Samples: X=100, Y=100
Number of Test Samples: X=100, Y=100


In [12]:
testing_optimizers = ['Adam'] #['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['relu'] #['tanh', 'relu', 'selu']

for optimizer in testing_optimizers:
    for activation in testing_activations:
        loss, accuracy, mae, mse = {}, {}, {}, {}
        train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}

        if optimizer == 'RAdam':
            score_returned, history = diff_optimizer(RAdam(lr=0.005), activation, label='RAdam')
        else:
            score_returned, history = diff_optimizer(optimizer, activation)


        bg = (score_returned[0], score_returned[1], score_returned[2], score_returned[3], history.get('loss'),\
          history.get('accuracy'), history.get('mae'), history.get('mae'))

        #Delete Existing Primary Keys and then write to DB
        deleteExistingPrimaryKeyDB(optimizer, activation, architecture, label, keywords)
        writeToDB(optimizer, activation, architecture, bg, label, keywords)

print ("Completed Execution for architecture={}".format(architecture))

Train on 1000 samples
Epoch 1/15
1000/1000 [==============================] - 1s 953us/sample - loss: 2.3024 - accuracy: 0.2010 - mae: 0.1800 - mse: 0.0900
Epoch 2/15
1000/1000 [==============================] - 0s 211us/sample - loss: 2.2622 - accuracy: 0.2140 - mae: 0.1791 - mse: 0.0892
Epoch 3/15
1000/1000 [==============================] - 0s 193us/sample - loss: 2.2198 - accuracy: 0.2140 - mae: 0.1782 - mse: 0.0884
Epoch 4/15
1000/1000 [==============================] - 0s 189us/sample - loss: 2.1638 - accuracy: 0.2140 - mae: 0.1769 - mse: 0.0873
Epoch 5/15
1000/1000 [==============================] - 0s 228us/sample - loss: 2.0914 - accuracy: 0.2140 - mae: 0.1750 - mse: 0.0860
Epoch 6/15
1000/1000 [==============================] - 0s 210us/sample - loss: 2.0034 - accuracy: 0.2140 - mae: 0.1725 - mse: 0.0845
Epoch 7/15
1000/1000 [==============================] - 0s 197us/sample - loss: 1.9038 - accuracy: 0.2140 - mae: 0.1693 - mse: 0.0830
Epoch 8/15
1000/1000 [==================